# python3的编码

正式讲述之前，我们先来看一段历史：

    因为计算机只能处理数字，如果要处理文本，就必须先把文本转换为数字才能处理。最早的计算机在设计时采用8个比特(bit)作为一个字节(byte)，所以，一个字节能表示的最大的整数就是255（二进制11111111=十进制255），如果要表示更大的整数，就必须用更多的字节。比如两个字节可以表示的最大整数是65535，4个字节可以表示的最大整数是4294967295。

    由于计算机是美国人发明的，因此，最早只有127个字符被编码到计算机里，也就是大小写英文字母、数字和一些符号，这个编码表被称为ASCII编码，比如大写字母A的编码是65，小写字母z的编码是122。

    但是要处理中文显然一个字节是不够的，至少需要两个字节，而且还不能和ASCII编码冲突，所以，中国制定了GB2312编码，用来把中文编进去。

    你可以想得到的是，全世界有上百种语言，日本把日文编到Shift_JIS里，韩国把韩文编到Euc-kr里，各国有各国的标准，就会不可避免地出现冲突，结果就是，在多语言混合的文本中，显示出来会有乱码。

    因此，Unicode应运而生。Unicode把所有语言都统一到一套编码里，这样就不会再有乱码问题了。

    Unicode标准也在不断发展，但最常用的是用两个字节表示一个字符（如果要用到非常偏僻的字符，就需要4个字节）。现代操作系统和大多数编程语言都直接支持Unicode，如我们要讲的python。
    
    看来是解决了问题，但是这时美国人又不愿意了：我原本表示一个字符只需要一个字节，现在倒好，为了适应你们一个字符增加了一个字节，如果我写的文本基本上全部是英文的话，用Unicode编码比ASCII编码需要多一倍的存储空间，在存储和传输上就十分不划算。
    
    于是，在美国人的牵头下，把Unicode编码转化为“可变长编码”的UTF-8编码。UTF-8编码把一个Unicode字符根据不同的数字大小编码成1-6个字节，常用的英文字母被编码成1个字节，汉字通常是3个字节，只有很生僻的字符才会被编码成4-6个字节。如果你要传输的文本包含大量英文字符，用UTF-8编码就能节省空间：

### 1.python3中文本字符串str和字节字符串是严格区分的

编码(encode)：将Unicode字符串（中的代码点)转换特定字符编码对应的字节串的过程和规则

解码(decode)：将特定字符编码的字节串转换为对应的Unicode字符串(中的代码点)的过程和规则

+ **在python3中阅读字符串时和存贮在磁盘上的字符串是以unicode编码对应计算机二进制的，要想转变为其他编码格式需要对其进行encode（）编码**

In [1]:
a= "哈哈"

In [2]:
a.encode("utf8")

b'\xe5\x93\x88\xe5\x93\x88'

In [6]:
type(a.encode("utf8"))

bytes

In [5]:
a.encode("gb2312")

b'\xb9\xfe\xb9\xfe'

```
(1)a.encode("utf8)为bytes类型,其值形式为 b'xxxxx'，其中 'xxxxx' 为一至多个转义的十六进制字符串（单个 x 的形式为：\x12，其中\x为小写的十六进制转义字符，12为二位十六进制数）组成的序列，每个十六进制字符串代表一个字节（八位二进制数，取值范围0-255），对于同一个字符串如果采用不同的编码方式生成bytes对象，就会形成不同的值.
(2)在utf-8编码中，每个汉字对应3个字节,所以两个汉字共有6个字节。这也符合上面的输出结果，且前后3个字节相同
```

In [3]:
a.decode()

AttributeError: 'str' object has no attribute 'decode'

In [4]:
a.encode("utf8").decode("utf8")

'哈哈'

**总结:**
```
(1)a是一个字符串，其编码格式是unicode，类型为str字符串，但是将其进行编码成utf8格式，会出现开头为b的字节字符串，其类型为 bytes ,故可以发现两者是不同的。
(2)字符串str本身没有decode这个方法，因为其本身就是unicode的。
(3)将a先解码在编码就可以变为原来的字符串，说明，在python3中编码必须经过unicode进行转化。
```

### 2.关于pthon3中对于文件的读取写入与储存

#### 第一部分：python读入文件的编码方式

#### （1）文件储存为utf8格式

首先自己创建一个文档，定义为“test1(utf8).txt”，内容为"哈哈123abc"，我们来看一下，python3是以什么方式读取写入文件的(文件格式可以在文件—另存为中选择)

注：显示的为当下的编码类型

![jupyter](https://i0.hdslb.com/bfs/album/da58775f166f4063ada67ba2847d68e1a744e7c9.png@518w_1e_1c.png)

In [46]:
f11 = open("test1(utf8).txt","r").read()

In [47]:
f11

'鍝堝搱abc123'

In [48]:
f12 = open("test1(utf8).txt","r" , encoding="utf8").read()

In [49]:
f12

'哈哈abc123'

In [50]:
f13 = open("test1(utf8).txt","r" , encoding="gbk").read()

In [51]:
f13

'鍝堝搱abc123'

由上面结果可以得出结论：
```
(1)python3先读入一行命令(本质上是一行字符串)，根据python3默认unicode编码找到字符串对应功能，当阅读到open时知道我现在要打开一个文件。而encoding的作用是指示python3以什么方式解码所要读取信息，使之成为unicode。
(2)科普:所谓乱码本质上是系统（windows默认）编码与所提供字符的编码不一致导致的（windows默认ANSI），ANSI在世界由地方不同而不同，在简体中文Windows操作系统中，ANSI 编码代表 GBK 编码；在繁体中文Windows操作系统中，ANSI编码代表Big5；在日文Windows操作系统中，ANSI 编码代表 Shift_JIS 编码，等等。
(3)python3默认是以ANSI读取存在系统的字节信息，然后内部自动解码成unicode。之前说过python是区分字符unicode和字节字符（utf8\gbk等）的，当不指明encoding正确读取方式时，就会出现乱码。
这里有一个疑问：ANSI到底是读取方式还是解码方式?换句话说python3读取字节信息时是先把所有内容以encoding方式读取完存在系统内，然后再自动解码成unicode还是直接以编码方式解码成unicode?如果是第二种情况为什么是参数是encoding而不是decoding?
```

#### （2）文件储存为gbk(ANSI)格式

再创建一个文档，定义为“test2(gbk).txt”，内容也为"哈哈123abc"。

注：显示的为当下的编码类型

![jupyter](https://i0.hdslb.com/bfs/album/98b45e8e699c4c52267617f8bb0e25440fc0ccc1.png@518w_1e_1c.png)

In [52]:
f21 = open("test2(gbk).txt","r").read()

In [53]:
f21

'哈哈abc123'

In [54]:
f22 = open("test2(gbk).txt","r", encoding="utf8").read()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb9 in position 0: invalid start byte

In [55]:
f23 = open("test2(gbk).txt","r", encoding="gb2312").read()

In [56]:
f23

'哈哈abc123'

+ **总结：**
```
(1)在gbk格式下的txt文件，在不指明编码方式下就可以正确读取为字符串，印证了上述看法。
(2)在指明为utf8下会报错，但在上面未报错，只是出现乱码，总之不能正确解码。
(3)当用jupyter打开"test2(gbk).txt"时，会报错。猜想：jupyter是默认以utf8方式读取系统的信息的，这与python默认ANSI方式不同.
```

**第二部分：python保存文件的编码格式**

In [30]:
with open("test3(ANSI).txt", "w") as f:
    f.write("哈哈123abc")

In [32]:
with open("test4(utf8).txt", "w",  encoding="utf8") as f:
    f.write("哈哈123abc")

![jupyter](https://i0.hdslb.com/bfs/album/eea94e602dbc432f57a3f29c7e29b3efa1715c81.png@518w_1e_1c.png)

![jupyter](https://i0.hdslb.com/bfs/album/c76194c574c637cac3b2c4eb007318d477f1aa00.png@518w_1e_1c.png)

由上图可知：

（1）默认是以ANSI进行保存，在使用encoding（）指明编码方式下，会使用指示的方式

（2）说明在python3中，会自动将unicode编码输出，encoding的作用很大

### 3.关于网页爬取编码 r.text 与 r.content

(1)对requests获取的原始数据，有三种获取形式：
```
    1.是r.content（万能，但须解码）           2.r.text              3.resp.json()
    text -- 审查元素中的全部，类型 str （直接表示为unicode）
    content -- 以b开头的审查元素中的全部内容，类型 bytes(含有编码形式的，故有时需要先解码才能正常显示文本，图片则不需要解码)
    json() -- 是针对json格式数据，可以直接生成字典进行数据提取
```
(2)关于 content 获取二进制的形式，如果需要变为text 这种获取的格式，可以使用 r.content.decode('utf-8') 

# python3编码总结：
```
(1)python以声明的编码方式读取文件字节（默认系统gbk）之后自动解码为unicode,显示为字符串，故字符串无decode用法。但是若将字符串编码为字节字符串(有编码格式的)再变为原字符串则需先decode再encode。
(2) windows系统下，无论是python还是其他默认保存与读取为gbk格式，除非在保存时指定编码方式，encoding=utf8。
(3)在python中，从网上获取r.text类型为str（unicode）无decode，但是r.content是二进制形式（带编码类型）需先解码才可以==r.text。
(4)爬虫中的尝试编码格式r.encoding = r.apparent_encoding  使得返回的内容解码正确（命令要打正确）才能在python中变为unicode。
```

--------------------------------------------------------------------------------这是一个正经的分隔符(ﾟДﾟ≡ﾟдﾟ)!?----------------------------------------------------------------------------------

-----------------------------------------------------------------------------一脸正气(●￣(ｴ)￣●)------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------确实非常正经( ♥д♥)-----------------------------------------------------------------------------------------------------------

# python--数据类型bytes

概述：bytes 是 Python 3.x 新增的类型，在 Python 2.x 中是不存在的。

并且在Python3以后，字符串和bytes类型彻底分开了,字节串（bytes）和字符串（string）的对比：
+ 字符串由若干个字符组成，以字符为单位进行操作；字节串由若干个字节组成，以字节为单位进行操作。
+ 字节串和字符串除了操作的数据单元不同之外，它们支持的所有方法都基本相同。
+ 字节串和字符串都是不可变序列，不能随意增加和删除数据。

bytes 只负责以字节序列的形式（二进制形式）来存储数据，至于这些数据到底表示什么内容（字符串、数字、图片、音频等），完全由程序的解析方式决定。如果采用合适的字符编码方式（字符集），字节串可以恢复成字符串；反之亦然，字符串也可以转换成字节串。

说白了，bytes 只是简单地记录内存中的原始数据，至于如何使用这些数据，bytes 并不在意，你想怎么使用就怎么使用，bytes 并不约束你的行为。

bytes 类型的数据非常适合在互联网上传输，可以用于网络通信编程；bytes 也可以用来存储图片、音频、视频等二进制格式的文件。

字符串和 bytes 存在着千丝万缕的联系，我们可以通过字符串来创建 bytes 对象，或者说将字符串转换成 bytes 对象。有以下三种方法可以达到这个目的：
+ 1.如果字符串的内容都是 ASCII 字符(英文字母和其他特殊符号)，那么直接在字符串前面添加**b**前缀就可以转换成 bytes。
+ 2.bytes 是一个类，调用它的构造方法，也就是 bytes()，可以将字符串按照指定的字符集转换成 bytes；b = bytes('string',encoding='编码类型')
+ 3.字符串本身有一个 encode() 方法，该方法专门用来将字符串按照指定的字符集转换成对应的字节串；

In [41]:
# 1.通过b前缀将ASCII字符串转换成 bytes
b1 = b'nihao'
print(b1)
print(type(b1))

b'nihao'
<class 'bytes'>


In [36]:
# 2.通过bytes() 方法指定字符集
b2 = bytes("哈哈",encoding="utf8")
print(b2)
print(type(b2))

b'\xe5\x93\x88\xe5\x93\x88'
<class 'bytes'>


In [38]:
# 3.通过 encode() 方法将字符串转换成 bytes
b3 = "哈哈"
b3.encode("utf8")

b'\xe5\x93\x88\xe5\x93\x88'

这里要注意：因为python3环境中字符串采用unicode编码，所以使用encode()方法具有可行性(因为这个方法前提是将unicode字符串转换特定字符编码)

下面我们来看三种情况下的输出结果:

+ 对于ASCII字符,print输出的是字符本身。

+ 对于非 ASCII 字符，print 输出的是它的字符编码值（十六进制形式），而不是字符本身。非 ASCII 字符一般占用两个字节以上的内存(ASCII:一个字符占一个字节,gbk、unicode、utf-8:一个字符占用两个以上字节)

+ bytes 是按照单个字节来处理数据的，不能一次处理多个字节。

**补充**：bytes的值形式为 b'xxxxx'。对于非 ASCII 字符：其中 'xxxxx' 为一至多个转义的十六进制字符串（单个 x 的形式为：\x12，其中\x为小写的十六进制转义字符，12为二位十六进制数）组成的序列，每个十六进制字符串代表一个字节（八位二进制数，取值范围0-255），对于同一个字符串如果采用不同的编码方式生成bytes对象，就会形成不同的值.

另外，bytes 类也有一个 decode() 方法，通过该方法可以将 bytes 对象转换为字符串:

In [42]:
b4 = b'\xe5\x93\x88\xe5\x93\x88'
b4.decode()

'哈哈'